[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ronghuaxueleng/colab-stable-diffusion-webui/blob/main/stable_diffusion_1_5_webui_colab_zh.ipynb)


# Stable Diffusion 1.5 WebUI Colab 简体中文

### 使用流程

1. 到**下载模型**, **Extensions**, **其他设置**进行设置
2. 左上角工具列 > 执行阶段 > 变更执行阶段类型 > 硬件加速器改为 GPU
3. 首次执行步骤：左上角工具列 > 执行阶段 > 全部执行
   非首次执行步骤：**前置步骤**->**Extensions**->**其他设置**->**启动WebUI**
4. 到**启动 WebUI** 点进 public URL

### 参考资料
https://colab.research.google.com/drive/1lekLF7iib6M1R-NCylS0VMTF4wve-XuV

### 更新时间
2023/04/10

## 1 - 前置步骤

In [ ]:
# @title 1.1 下载 stable-diffusion-webui
%cd /content
repository_url = "https://github.com/camenduru/stable-diffusion-webui"  # @param {type: "string"}
webui_branch = "v2.1"  # @param ["v2.1", "master"]

! git clone {repository_url} SDWCT

%cd /content/SDWCT
! git checkout -f {webui_branch}

In [ ]:
# @title 1.2 建立云端资料夹
%cd /content
# @markdown 云端资料夹名称
data_dir_name = "Stable_Diffusion_1_5_WebUI_Colab_zh_CN"  # @param {type:"string"}
data_dir_base = "/content/drive/MyDrive"
data_dir = f"{data_dir_base}/{data_dir_name}"

import os

drive_path = "/content/drive"
from google.colab import drive

try:
    drive.mount(drive_path, force_remount=True)
    os.makedirs(data_dir, exist_ok=True)
except:
    print('Mount Error')
    raise

models_path = f"{data_dir}/models"
output_path = f"{data_dir}/outputs"
config_path = f"{data_dir}/config"
scripts_path = f"{data_dir}/scripts"
extensions_file_path = f"{data_dir}/extensions.txt"

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(config_path, exist_ok=True)
os.makedirs(scripts_path, exist_ok=True)
os.makedirs(f"{models_path}/Stable-diffusion", exist_ok=True)
os.makedirs(f"{models_path}/VAE", exist_ok=True)
os.makedirs(f"{models_path}/embeddings", exist_ok=True)
os.makedirs(f"{models_path}/hypernetworks", exist_ok=True)
os.makedirs(f"{models_path}/Lora", exist_ok=True)
os.makedirs(f"{models_path}/ESRGAN", exist_ok=True)
os.makedirs(f"{models_path}/ControlNet", exist_ok=True)

! cp -Rf SDWCT/scripts/* {scripts_path}
! rm -Rf SDWCT/scripts && ln -s {scripts_path} SDWCT/scripts

for dir in os.listdir(models_path):
    if dir == "embeddings":
        ! rm -Rf SDWCT/embeddings && ln -s {models_path}/embeddings SDWCT/embeddings
    elif dir != "ControlNet":
        ! rm -Rf SDWCT/models/{dir} && ln -s {models_path}/{dir} SDWCT/models/{dir}

! rm -Rf SDWCT/outputs && ln -s {data_dir}/outputs SDWCT/outputs

for filename in ["config.json", "ui-config.json", "styles.csv", "extensions.txt"]:
    ! rm -f SDWCT/{filename}
    filepath = f"{config_path}/{filename}"
    if not os.path.exists(filepath):
        if filename.endswith(".json"):
            with open(filepath, mode="w") as f:
                f.write("{}")
        else:
            ! touch {config_path}/{filename}
    ! ln -s {config_path}/{filename} SDWCT/{filename}

In [ ]:
# @title 1.3 设置python环境
! wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
! chmod +x mini.sh
! bash ./mini.sh -b -f -p /usr/local
! wget -O restore_env.sh https://raw.fastgit.org/ronghuaxueleng/colab-stable-diffusion-webui/main/restore_env.sh
! chmod +x restore_env.sh
! bash ./restore_env.sh

## 2 - 下载模型

In [ ]:
# @title 2.1 下载SD模型
# @markdown 需要下载SD模型组合包吗？
need_to_download_sd = True # @param {type:"boolean"}
# @markdown 选择SD模型组合包
model_package = "ChilloutMix_Ni_fix" # @param ["ChilloutMix_Ni_fix", "其他(下方填写)"] 
# @markdown 其他SD模型下载网址
sd_url = "" #@param {type:"string"}
vae_url = "" #@param {type:"string"}
# @markdown ---
# @markdown - 除了利用Colab下载模型，推荐使用云端硬盘以节省空间。
# @markdown  - [[AI绘图] WebUI Colab玩家小技巧：使用云端硬盘节省空间](https://home.gamer.com.tw/artwork.php?sn=5677784)
# @markdown - Anything, AbyssOrangeMix, Counterfeit 开启 VAE 图片色彩比较靓丽
# @markdown  - SD VAE > anime.vae.pt
# @markdown - CamelliaMix_2.5D 开启 VAE 图片色彩比较靓丽
# @markdown  - SD VAE > vae-ft-mse-840000-ema-pruned.safetensors
# @markdown - PastalMix 和 ChilloutMix 不需要设置 VAE
# @markdown  - SD VAE > None

sd_urls = [
    "https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt",
    "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors"
]
vae_urls = []
if model_package == "ChilloutMix_Ni_fix":
    sd_urls.append("https://civitai.com/api/download/models/11745")
elif model_package == "其他(下方填写)":
    sd_urls.append(sd_url)
    vae_urls.append(vae_url)

! git clone https://huggingface.co/embed/negative {models_path}/embeddings/negative
! git clone https://huggingface.co/embed/lora {models_path}/Lora/positive
! wget https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -O {models_path}/ESRGAN/4x-UltraSharp.pth
! wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O {scripts_path}/run_n_times.py

if sd_urls != [] and need_to_download_sd == True:
    %cd {models_path}/Stable-diffusion
    for sd_url in sd_urls:
        if sd_url != "":
            ! wget -nc --content-disposition {sd_url}

if vae_urls != [] and need_to_download_sd == True:
    %cd {models_path}/VAE
    for vae_url in vae_urls:
        if vae_url != "":  
            ! wget -nc --content-disposition {vae_url}

In [ ]:
# @title 2.2 下载 Embedding + Hypernetwork + LoRA

# @markdown Embedding
EasyNegative = True # @param {type:"boolean"}
embedding_urls = ""  # @param {type:"string"}

# @markdown Hypernetwork
hypernetwork_urls = ""  # @param {type:"string"}

# @markdown LoRA
koreanDollLikeness_v10 = True # @param {type:"boolean"}
koreanDollLikeness_v15 = True # @param {type:"boolean"}
taiwanDollLikeness_v10 = True # @param {type:"boolean"}
japaneseDollLikeness_v10 = True # @param {type:"boolean"}
lora_urls = ""  # @param {type:"string"}

# @markdown ---
# @markdown - [EasyNegative](https://civitai.com/models/7808/easynegative)
# @markdown   - 泛用负面 embedding
# @markdown   - 触发词为 EasyNegative, 加在负面 prompt
# @markdown - DollLikeness 系列
# @markdown   - 触发词为 woman, girl
# @markdown   - 模型权重建议为 0.5
# @markdown - 网址之间用英文逗号间隔。例：`url1`, `url2`, `url3`
# @markdown - [LoRA预览图](https://i.imgur.com/XooSV8p.png)请自行添加，格式为：`模型名称.png`

embeddings = []
embedding_urls = list(map(str.strip, embedding_urls.split(",")))
if EasyNegative == True:
    embeddings.append("https://civitai.com/api/download/models/9208")
if embedding_urls != ['']:
    embeddings = embeddings + embedding_urls

if embeddings != []:
    %cd {models_path}/embeddings
    for embedding in embeddings:
        ! wget -nc --content-disposition {embedding}

hypernetworks = []
hypernetwork_urls = list(map(str.strip, hypernetwork_urls.split(",")))
if hypernetwork_urls != ['']:
    hypernetworks = hypernetworks + hypernetwork_urls

if hypernetworks != []:
    %cd {models_path}/hypernetworks
    for hypernetwork in hypernetworks:
        ! wget -nc --content-disposition {hypernetwork}

loras = []
lora_urls = list(map(str.strip, lora_urls.split(",")))
if koreanDollLikeness_v10 == True:
    loras.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/LoRA/DollLikeness/koreanDollLikeness_v10.safetensors")
if koreanDollLikeness_v15 == True:
    loras.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/LoRA/DollLikeness/koreanDollLikeness_v15.safetensors")
if taiwanDollLikeness_v10 == True:
    loras.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/LoRA/DollLikeness/taiwanDollLikeness_v10.safetensors")
if japaneseDollLikeness_v10 == True:
    loras.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/LoRA/DollLikeness/japaneseDollLikeness_v10.safetensors")
if lora_urls != ['']:
    loras = loras + lora_urls

if loras != []:
    %cd {models_path}/Lora
    for lora in loras:
        ! wget -nc --content-disposition {lora}

## 3 - Extensions

In [ ]:
# @title 3.1 下载 Extensions
deforum = True # @param {type:"boolean"}
localization_zh_CN = True # @param {type:"boolean"}
image_browser = True # @param {type:"boolean"}
huggingface = True # @param {type:"boolean"}
civitai_browser = True # @param {type:"boolean"}
additional_networks = True # @param {type:"boolean"}
controlnet = True # @param {type:"boolean"}
openpose_editor = True # @param {type:"boolean"}
depth_lib = True # @param {type:"boolean"}
posex = True # @param {type:"boolean"}
open_pose_editor_4_3d = False # @param {type:"boolean"}
tunnels = False # @param {type:"boolean"}
batchlinks = False # @param {type:"boolean"}
catppuccin = False # @param {type:"boolean"}
locon = False # @param {type:"boolean"}
rembg = True # @param {type:"boolean"}
two_shot = True # @param {type:"boolean"}
pnginfo = True # @param {type:"boolean"}
extension_urls = ""  # @param {type:"string"}

# @markdown ---
# @markdown - [[AI绘图] Stable Diffusion WebUI Colab TW Extensions整理](https://home.gamer.com.tw/artwork.php?sn=5678466)
# @markdown - 网址之间用英文逗号隔开。例：`url1`, `url2`, `url3`
# @markdown - 如果使用WebUI內建方法下载 Extension，仍然要执行 `下载 Extensions` 以记录下载的 Extension。
# @markdown - 如果不想再下载某个 Extension，请删除 `config/extensions.txt` 上对应的 GitHub 网址。


extensions_pair = [   
            (deforum, "https://github.com/deforum-art/deforum-for-automatic1111-webui"),
            (localization_zh_CN, "https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN"),
            (image_browser, "https://github.com/camenduru/stable-diffusion-webui-images-browser"),
            (huggingface, "https://github.com/camenduru/stable-diffusion-webui-huggingface"),
            (civitai_browser, "https://github.com/camenduru/sd-civitai-browser"),
            (additional_networks, "https://github.com/kohya-ss/sd-webui-additional-networks"),
            (controlnet, "https://github.com/Mikubill/sd-webui-controlnet"),
            (openpose_editor, "https://github.com/fkunn1326/openpose-editor"),
            (depth_lib, "https://github.com/jexom/sd-webui-depth-lib"),
            (posex, "https://github.com/hnmr293/posex"),
            (open_pose_editor_4_3d, "https://github.com/nonnonstop/sd-webui-3d-open-pose-editor"),
            (tunnels, "https://github.com/camenduru/sd-webui-tunnels"),
            (batchlinks, "https://github.com/etherealxx/batchlinks-webui"),
            (catppuccin, "https://github.com/camenduru/stable-diffusion-webui-catppuccin"),
            (locon,"https://github.com/KohakuBlueleaf/a1111-sd-webui-locon"),
            (rembg, "https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg"),
            (two_shot, "https://github.com/ashen-sensored/stable-diffusion-webui-two-shot"),
            (pnginfo, "https://github.com/camenduru/sd_webui_stealth_pnginfo")
          ]

with open('/content/SDWCT/extensions.txt', 'r') as f:
    extensions_txt = f.read()
extensions_txt = list(map(str.strip, extensions_txt.split("\n")))

extension_urls = list(map(str.strip, extension_urls.split(",")))

extensions = []
for url in extensions_txt:
    if url != '' and url not in extensions:
        extensions.append(url)   
for name, url in extensions_pair:
    if name == True and url not in extensions:
        extensions.append(url)
for url in extension_urls:
    if url != '' and url not in extensions:
        extensions.append(url)

import os

if extensions != []:
    %cd /content/SDWCT/extensions
    for extension in extensions:
        extension_name, _ = os.path.splitext(extension.split("/")[-1])
        if not os.path.exists(extension_name):
            ! git clone {extension}

%cd /content/SDWCT
! git reset --hard
! git -C /content/SDWCT/repositories/stable-diffusion-stability-ai reset --hard

%cd /content/SDWCT/extensions
if os.path.exists("sd-webui-controlnet") and not os.path.islink("sd-webui-controlnet/models"):
    ! cp sd-webui-controlnet/models/cldm_v15.yaml {models_path}/ControlNet/cldm_v15.yaml
    ! cp sd-webui-controlnet/models/cldm_v21.yaml {models_path}/ControlNet/cldm_v21.yaml
    ! cp sd-webui-controlnet/models/image_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_depth_sd14v1.yaml
    ! cp sd-webui-controlnet/models/image_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_keypose_sd14v1.yaml
    ! cp sd-webui-controlnet/models/image_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_openpose_sd14v1.yaml
    ! cp sd-webui-controlnet/models/image_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_seg_sd14v1.yaml
    ! cp sd-webui-controlnet/models/sketch_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_canny_sd14v1.yaml
    ! cp sd-webui-controlnet/models/sketch_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_sketch_sd14v1.yaml
    ! cp sd-webui-controlnet/models/t2iadapter_color_sd14v1.yaml {models_path}/ControlNet/t2iadapter_color_sd14v1.yaml
    ! cp sd-webui-controlnet/models/t2iadapter_style_sd14v1.yaml {models_path}/ControlNet/t2iadapter_style_sd14v1.yaml
    ! rm -Rf sd-webui-controlnet/models && ln -s {models_path}/ControlNet sd-webui-controlnet/models

if os.path.exists("a1111-sd-webui-tagcomplete"):
    ! git clone -b tags https://github.com/ronghuaxueleng/colab-stable-diffusion-webui.git my_tags0424
    ! cp -rf my_tags0424/* ./a1111-sd-webui-tagcomplete/tags
    ! rm -rf my_tags0424
    ! ls ./a1111-sd-webui-tagcomplete/tags

if os.path.exists("Stable-Diffusion-Webui-Prompt-Translator"):
    ! echo -n '{"baidu": {"is_default": true, "app_id": "20230425001655753", "app_key": "aZUcg_Z6kDGq0T2ix_4Y"}}' > Stable-Diffusion-Webui-Prompt-Translator/prompt_translator.cfg

%cd /content/SDWCT/extensions
! echo -n '' > /content/SDWCT/extensions.txt
for dir in next(os.walk("/content/SDWCT/extensions"))[1]:
    %cd {dir}
    if os.path.exists(".git"):
        ! git config --get remote.origin.url >> /content/SDWCT/extensions.txt
    %cd ..

In [ ]:
# @title 3.2 下载 ControlNet 模型
# @markdown ControlNet
control_canny = False # @param {type:"boolean"}
control_depth = False # @param {type:"boolean"}
control_hed = False # @param {type:"boolean"}
control_mlsd = False # @param {type:"boolean"}
control_normal = False # @param {type:"boolean"}
control_openpose = True # @param {type:"boolean"}
control_scribble = False # @param {type:"boolean"}
control_seg = False # @param {type:"boolean"}
# @markdown T2I Adapter
t2iadapter_canny = False # @param {type:"boolean"}
t2iadapter_color = False # @param {type:"boolean"}
t2iadapter_depth = False # @param {type:"boolean"}
t2iadapter_keypose = False # @param {type:"boolean"}
t2iadapter_openpose = False # @param {type:"boolean"}
t2iadapter_seg = False # @param {type:"boolean"}
t2iadapter_sketch = False # @param {type:"boolean"}
t2iadapter_style = False # @param {type:"boolean"}
# @markdown ---
# @markdown - 除了利用Colab下载模型，推荐使用云端硬盘以节省空间。
# @markdown  - [[AI绘图] WebUI Colab玩家小技巧：使用使用云端硬盘以节省空间](https://home.gamer.com.tw/artwork.php?sn=5677784)
# @markdown - 多重 ControlNet 请到 Settings > ControlNet > Multi ControlNet 设置
# @markdown - 参考资料
# @markdown  - https://github.com/Mikubill/sd-webui-controlnet
# @markdown  - https://github.com/lllyasviel/ControlNet
# @markdown  - https://github.com/TencentARC/T2I-Adapter
# @markdown  - [Stable diffusion ControlNet使用心得](https://home.gamer.com.tw/artwork.php?sn=5662905)

import os

control_net_models = []
t2iadapter_models = []

if control_canny == True:
    control_net_models.append("canny")
if control_depth == True:
    control_net_models.append("depth")
if control_hed == True:
    control_net_models.append("hed")
if control_mlsd == True:
    control_net_models.append("mlsd")
if control_normal == True:
    control_net_models.append("normal")
if control_openpose == True:
    control_net_models.append("openpose")
if control_scribble == True:
    control_net_models.append("scribble")
if control_seg == True:
    control_net_models.append("seg")

if t2iadapter_canny == True:
    t2iadapter_models.append("canny")
if t2iadapter_color == True:
    t2iadapter_models.append("color")
if t2iadapter_depth == True:
    t2iadapter_models.append("depth")
if t2iadapter_keypose == True:
    t2iadapter_models.append("keypose")
if t2iadapter_openpose == True:
    t2iadapter_models.append("openpose")
if t2iadapter_seg == True:
    t2iadapter_models.append("seg")
if t2iadapter_sketch == True:
    t2iadapter_models.append("sketch")
if t2iadapter_style == True:
    t2iadapter_models.append("style")

%cd /content/SDWCT/extensions

if os.path.exists('sd-webui-controlnet'):
    %cd sd-webui-controlnet/models
    for control_net_model in control_net_models:
        ! wget -nc --content-disposition https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_{control_net_model}-fp16.safetensors
    for t2iadapter_model in t2iadapter_models:
        ! wget -nc --content-disposition https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_{t2iadapter_model}_sd14v1.pth        

In [ ]:
# @title 3.3 下载档案到指定文件夹
target_dir = "" # @param {type:"string"} 
file_urls = "" # @param {type:"string"}
# @markdown ---
# @markdown - 将制定文件夹填在 `target_dir`，档案的下载地址填在 `file_urls`。
# @markdown - 网址之间用英文逗号隔开。例：`url1`, `url2`, `url3`
file_urls = list(map(str.strip, file_urls.split(",")))

import os
if os.path.exists(os.path.split(target_dir)[0]):
    os.makedirs(target_dir, exist_ok=True)
    %cd {target_dir}
    for url in file_urls:
        if url != '':
            ! wget -nc --content-disposition {url}

In [ ]:
# @title 3.4 自定义资料夹连接
google_drive_dir = "" # @param {type:"string"} 
webui_dir = "" # @param {type:"string"}
# @markdown ---
# @markdown - 将云端硬盘资料夹填在 `google_drive_dir`，WebUI资料夹填在 `webui_dir`。

import os
if os.path.exists(os.path.split(webui_dir)[0]):
    os.makedirs(google_drive_dir, exist_ok=True)
    if os.path.exists(webui_dir):
        ! cp -Rf {webui_dir}/* {google_drive_dir}
    else:
        os.makedirs(webui_dir, exist_ok=True)
    ! rm -Rf {webui_dir} && ln -s {google_drive_dir} {webui_dir}

## 4 - 其他设置

In [ ]:
# @title 4.1 config.json
recommended_settings = True # @param {type:"boolean"}

import json

def read_config(filename):
  with open(filename, "r") as f:
    config = json.load(f)
  return config

def write_config(filename, config):
  with open(filename, "w") as f:
    json.dump(config, f, indent=4)

if recommended_settings == True:
    %cd /content/SDWCT/extensions
    config_file = f"{config_path}/config.json"
    config = read_config(config_file)
    if "CLIP_stop_at_last_layers" not in config or config["CLIP_stop_at_last_layers"] == 1:
        config["CLIP_stop_at_last_layers"] = 2
    if "quicksettings" not in config or config["quicksettings"] == "sd_model_checkpoint":
        config["quicksettings"] = "sd_model_checkpoint, sd_vae"    
    if os.path.exists("sd-webui-controlnet") and ("control_net_max_models_num" not in config or config["control_net_max_models_num"] == 1): 
        config["control_net_max_models_num"] = 3
    if os.path.exists("stable-diffusion-webui-localization-zh_CN"):
        if not os.path.exists("sd-webui-bilingual-localization") and ("localization" not in config or config["localization"] == "None"): 
            config["localization"] = "zh_CN"
        elif os.path.exists("sd-webui-bilingual-localization") and ("bilingual_localization_file" not in config or config["bilingual_localization_file"] == "None"):
            config["localization"] = "None"
            config["bilingual_localization_file"] = "zh_CN"
    if os.path.exists("a1111-sd-webui-tagcomplete"):
       config["tac_tagFile"] = "zh_cn.csv"
       config["tac_extra.extraFile"] = "zh_cn.csv"
    write_config(config_file, config)  

In [ ]:
# @title 4.2 命令行参数
medvram = False # @param {type:"boolean"}
lowram = False # @param {type:"boolean"}
# @markdown SSH Tunnel
ssh_tunnel = "gradio" # @param ["gradio", "ngrok(推荐，需填写ngrok_authtoken)", "cloudflared", "localhost.run", "remote.moe", "googleusercontent.com", "multiple"]
ngrok_authtoken = "" # @param {type:"string"}
# @markdown ---
# @markdown ngrok_authtoken 填写方法
# @markdown - 註冊 [ngrok](https://ngrok.com/) 账号
# @markdown - Your Authtoken > Copy
# @markdown - ssh_tunnel 选 ngrok > 贴上 ngrok_authtoken
# @markdown - 等 local URL 出来之后再点 ngrok URL

import os
from google.colab.output import eval_js

! sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/SDWCT/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/SDWCT/launch.py
! sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' /content/SDWCT/modules/shared.py

os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")
os.environ["COMMANDLINE_ARGS"] = "--xformers \
                  --no-hashing --enable-insecure-extension-access \
                  --disable-safe-unpickle --opt-channelslast --theme dark"

if medvram == True:
    os.environ["COMMANDLINE_ARGS"] += " --medvram"
if lowram == True:
    os.environ["COMMANDLINE_ARGS"] += " --lowram"

if ssh_tunnel == "gradio":
    os.environ["COMMANDLINE_ARGS"] += " --share"
elif ssh_tunnel == "ngrok(推薦，需填寫ngrok_authtoken)":
    os.environ["COMMANDLINE_ARGS"] += (" --ngrok " + ngrok_authtoken)
elif ssh_tunnel == "cloudflared":
    os.environ["COMMANDLINE_ARGS"] += " --share --cloudflared"
elif ssh_tunnel == "localhost.run":
    os.environ["COMMANDLINE_ARGS"] += " --share --localhostrun"
elif ssh_tunnel == "remote.moe":
    os.environ["COMMANDLINE_ARGS"] += " --share --remotemoe"
elif ssh_tunnel == "googleusercontent.com":
    os.environ["COMMANDLINE_ARGS"] += " --share --googleusercontent"
elif ssh_tunnel == "multiple":
    os.environ["COMMANDLINE_ARGS"] += " --multiple"

## 5 - 启动 WebUI

In [ ]:
# @title 5.1 启动 WebUI
# @markdown - 点选 public URL 开启 WebUI<br>
# @markdown ![](https://i.imgur.com/y3xGiIX.png)

%cd /content/SDWCT/
! python launch.py